# 11. Annotation

In [1]:
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
import scanpy as sc
import pandas as pd
import numpy as np
import os
from scipy.sparse import csr_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import celltypist
from celltypist import models
import scvi

/Users/adrian/opt/anaconda3/envs/mamba/envs/simpleaf/lib/python3.10/site-packages/scvi/__init__.py:31: DeprecationWarning: scvi is deprecated, please uninstall scvi via `pip uninstall scvi` and install the new scvi-tools package at github.com/YosefLab/scvi-tools
  warnings.warn(deprecation_msg, DeprecationWarning)


In [3]:
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)

In [4]:
path_query_data = "../../0_outs/sample-SRR15058452/SRR15058452_dimensionality_reduction.h5ad"
dir_reference_model = "./scvi_reference_model"
path_annotations = "./cell_type_annotations.csv"

In [5]:
sc.set_figure_params(figsize=(5, 5))

/Users/adrian/opt/anaconda3/envs/mamba/envs/simpleaf/lib/python3.10/site-packages/scanpy/_settings.py:447: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  IPython.display.set_matplotlib_formats(*ipython_format)


In [6]:
adata_complete = sc.read_h5ad(path_query_data)

In [7]:
#adata = adata_complete[adata_complete.obs["sample"] == "SRR15058452", :].copy()
adata = adata_complete.copy()

In [8]:
ct_annotations = pd.read_csv(path_annotations, index_col=0)

In [9]:
ct_annotations.head(3)

,cell_type
TTCGCAACAATAATGG,Early Lymphoid
ACTCGCGCAAACTGTT,CD14+ Mono
GACTATTCATGTCGCG,Naive CD20+ B


In [10]:
adata.obs.head(3)

,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_20_genes,total_counts_mt,log1p_total_counts_mt,pct_counts_mt,total_counts_ribo,log1p_total_counts_ribo,pct_counts_ribo,total_counts_hb,log1p_total_counts_hb,pct_counts_hb,outlier,mt_outlier,scDblFinder_score,scDblFinder_class,size_factors
AAACCCAAGACTACGG-1,3164,8.059908,11480.0,9.348449,16.925087,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,0.000418,1,0.955902
AAACCCAAGGGCAAGG-1,6184,8.729882,33525.0,10.420076,14.269948,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,0.000270,1,1.323977
AAACCCAAGTCCCGGT-1,4118,8.323366,18098.0,9.803612,16.178583,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,0.999847,2,1.060753


In [11]:
adata.obs.tail(3)

,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_20_genes,total_counts_mt,log1p_total_counts_mt,pct_counts_mt,total_counts_ribo,log1p_total_counts_ribo,pct_counts_ribo,total_counts_hb,log1p_total_counts_hb,pct_counts_hb,outlier,mt_outlier,scDblFinder_score,scDblFinder_class,size_factors
TTTGTTGTCACGATCA-1,2615,7.869402,8282.0,9.021960,16.179667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,0.002915,1,0.888677
TTTGTTGTCATGCATG-1,2774,7.928406,8367.0,9.032170,11.868053,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,0.000258,1,0.985588
TTTGTTGTCTCATGCC-1,5505,8.613594,27199.0,10.210972,14.923343,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,0.000221,1,1.274248


In [12]:
adata.obs.index = [x.split("-1")[0] for x in adata.obs.index]

In [13]:
adata.obs.head(3)

,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_20_genes,total_counts_mt,log1p_total_counts_mt,pct_counts_mt,total_counts_ribo,log1p_total_counts_ribo,pct_counts_ribo,total_counts_hb,log1p_total_counts_hb,pct_counts_hb,outlier,mt_outlier,scDblFinder_score,scDblFinder_class,size_factors
AAACCCAAGACTACGG,3164,8.059908,11480.0,9.348449,16.925087,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,0.000418,1,0.955902
AAACCCAAGGGCAAGG,6184,8.729882,33525.0,10.420076,14.269948,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,0.000270,1,1.323977
AAACCCAAGTCCCGGT,4118,8.323366,18098.0,9.803612,16.178583,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,0.999847,2,1.060753


In [14]:
adata.obs.tail(3)

,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_20_genes,total_counts_mt,log1p_total_counts_mt,pct_counts_mt,total_counts_ribo,log1p_total_counts_ribo,pct_counts_ribo,total_counts_hb,log1p_total_counts_hb,pct_counts_hb,outlier,mt_outlier,scDblFinder_score,scDblFinder_class,size_factors
TTTGTTGTCACGATCA,2615,7.869402,8282.0,9.021960,16.179667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,0.002915,1,0.888677
TTTGTTGTCATGCATG,2774,7.928406,8367.0,9.032170,11.868053,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,0.000258,1,0.985588
TTTGTTGTCTCATGCC,5505,8.613594,27199.0,10.210972,14.923343,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,0.000221,1,1.274248


In [15]:
#adata2 = adata[ct_annotations.index, :].copy()

# 11.4. Manual annotation

In [16]:
marker_genes = {
    "CD14+ Mono": ["FCN1", "CD14"],
    "CD16+ Mono": ["TCF7L2", "FCGR3A", "LYN"],
    "ID2-hi myeloid prog": [
        "CD14",
        "ID2",
        "VCAN",
        "S100A9",
        "CLEC12A",
        "KLF4",
        "PLAUR",
    ],
    "cDC1": ["CLEC9A", "CADM1"],
    "cDC2": [
        "CST3",
        "COTL1",
        "LYZ",
        "DMXL2",
        "CLEC10A",
        "FCER1A",
    ],  # Note: DMXL2 should be negative
    "Normoblast": ["SLC4A1", "SLC25A37", "HBB", "HBA2", "HBA1", "TFRC"],
    "Erythroblast": ["MKI67", "HBA1", "HBB"],
    "Proerythroblast": [
        "CDK6",
        "SYNGR1",
        "HBM",
        "GYPA",
    ],  # Note HBM and GYPA are negative markers
    "NK": ["GNLY", "NKG7", "CD247", "GRIK4", "FCER1G", "TYROBP", "KLRG1", "FCGR3A"],
    "ILC": ["ID2", "PLCG2", "GNLY", "SYNE1"],
    "Lymph prog": [
        "VPREB1",
        "MME",
        "EBF1",
        "SSBP2",
        "BACH2",
        "CD79B",
        "IGHM",
        "PAX5",
        "PRKCE",
        "DNTT",
        "IGLL1",
    ],
    "Naive CD20+ B": ["MS4A1", "IL4R", "IGHD", "FCRL1", "IGHM"],
    "B1 B": [
        "MS4A1",
        "SSPN",
        "ITGB1",
        "EPHA4",
        "COL4A4",
        "PRDM1",
        "IRF4",
        "CD38",
        "XBP1",
        "PAX5",
        "BCL11A",
        "BLK",
        "IGHD",
        "IGHM",
        "ZNF215",
    ],  # Note IGHD and IGHM are negative markers
    "Transitional B": ["MME", "CD38", "CD24", "ACSM3", "MSI2"],
    "Plasma cells": ["MZB1", "HSP90B1", "FNDC3B", "PRDM1", "IGKC", "JCHAIN"],
    "Plasmablast": ["XBP1", "RF4", "PRDM1", "PAX5"],  # Note PAX5 is a negative marker
    "CD4+ T activated": ["CD4", "IL7R", "TRBC2", "ITGB1"],
    "CD4+ T naive": ["CD4", "IL7R", "TRBC2", "CCR7"],
    "CD8+ T": ["CD8A", "CD8B", "GZMK", "GZMA", "CCL5", "GZMB", "GZMH", "GZMA"],
    "T activation": ["CD69", "CD38"],  # CD69 much better marker!
    "T naive": ["LEF1", "CCR7", "TCF7"],
    "pDC": ["GZMB", "IL3RA", "COBLL1", "TCF4"],
    "G/M prog": ["MPO", "BCL2", "KCNQ5", "CSF3R"],
    "HSC": ["NRIP1", "MECOM", "PROM1", "NKAIN2", "CD34"],
    "MK/E prog": [
        "ZNF385D",
        "ITGA2B",
        "RYR3",
        "PLCB1",
    ],  # Note PLCB1 is a negative marker
}

In [17]:
marker_genes_in_data = dict()
for ct, markers in marker_genes.items():
    markers_found = list()
    for marker in markers:
        if marker in adata.var.index:
            markers_found.append(marker)
    marker_genes_in_data[ct] = markers_found

In [18]:
adata.layers["counts"] = adata.X
adata.X = adata.layers["scran_normalization"]

In [19]:
adata

AnnData object with n_obs × n_vars = 8509 × 15954
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier', 'scDblFinder_score', 'scDblFinder_class', 'size_factors'
    var: 'gene_ids', 'feature_types', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'highly_deviant', 'binomial_deviance', 'highly_variable'
    uns: 'neighbors', 'pca', 'tsne', 'umap'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    varm: 'PCs'
    layers: 'analytic_pearson_residuals', 'counts', 'log1p_norm', 'scran_normalization', 'soupX_counts'
    obsp: 'connectivities', 'distances'

In [20]:
adata.var

,gene_ids,feature_types,mt,ribo,hb,n_cells_by_counts,mean_counts,log1p_mean_counts,pct_dropout_by_counts,total_counts,log1p_total_counts,n_cells,highly_deviant,binomial_deviance,highly_variable
Xkr4,ENSMUSG00000051951,Gene Expression,False,False,False,119,0.013669,0.013577,98.655671,121.0,4.804021,112,False,894.672728,False
Mrpl15,ENSMUSG00000033845,Gene Expression,False,False,False,2986,0.492544,0.400482,66.267510,4360.0,8.380457,2915,False,7947.242655,False
Lypla1,ENSMUSG00000025903,Gene Expression,False,False,False,3252,0.584727,0.460412,63.262540,5176.0,8.551981,3181,False,9082.554331,False
Tcea1,ENSMUSG00000033813,Gene Expression,False,False,False,4864,1.085404,0.734963,45.051966,9608.0,9.170455,4767,True,10656.224454,True
Rgs20,ENSMUSG00000002459,Gene Expression,False,False,False,33,0.004519,0.004509,99.627203,40.0,3.713572,32,False,431.840636,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Csprs,ENSMUSG00000062783,Gene Expression,False,False,False,109,0.012878,0.012796,98.768640,114.0,4.744932,105,False,1019.015364,False
Vamp7,ENSMUSG00000051412,Gene Expression,False,False,False,1145,0.160529,0.148876,87.065070,1421.0,7.259820,1119,False,4719.753989,False
Tmlhe,ENSMUSG00000079834,Gene Expression,False,False,False,868,0.160077,0.148486,90.194306,1417.0,7.257003,848,False,7426.144007,False
CAAA01147332.1,ENSMUSG00000095742,Gene Expression,False,False,False,89,0.010280,0.010228,98.994577,91.0,4.521789,86,False,712.104260,False


In [21]:
adata.var["highly_variable"] = adata.var["highly_deviant"]

In [22]:
adata.var["highly_variable"]

Xkr4              False
Mrpl15            False
Lypla1            False
Tcea1              True
Rgs20             False
                  ...  
Csprs             False
Vamp7             False
Tmlhe             False
CAAA01147332.1    False
AC149090.1         True
Name: highly_variable, Length: 15954, dtype: bool

In [23]:
sc.tl.pca(adata, n_comps=50, use_highly_variable=True)

In [24]:
sc.pp.neighbors(adata)

/Users/adrian/opt/anaconda3/envs/mamba/envs/simpleaf/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
sc.tl.umap(adata)

In [26]:
B_plasma_cts = [
    "Naive CD20+ B",
    "B1 B",
    "Transitional B",
    "Plasma cells",
    "Plasmablast",
]

In [29]:
for ct in B_plasma_cts:
    print(f"{ct.upper()}:")  # print cell subtype name
    sc.pl.umap(
        adata,
        color=marker_genes_in_data[ct],
        vmin=0,
        vmax="p99",  # set vmax to the 99th percentile of the gene count instead of the maximum, to prevent outliers from making expression in other cells invisible. Note that this can cause problems for extremely lowly expressed genes.
        sort_order=False,  # do not plot highest expression on top, to not get a biased view of the mean expression among cells
        frameon=False,
        cmap="Reds",  # or choose another color map e.g. from here: https://matplotlib.org/stable/tutorials/colors/colormaps.html
    )
    print("\n\n\n")  # print white space for legibility

NAIVE CD20+ B:


/Users/adrian/opt/anaconda3/envs/mamba/envs/simpleaf/lib/python3.10/site-packages/scanpy/plotting/_tools/scatterplots.py:163: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = copy(get_cmap(cmap))


ValueError: Could not broadast together arguments with shapes: [0, 1].